In [44]:
import pandas as pd

In [45]:
Humanness = [ 70, 476, 456, 439, 403, 383, 377, 376, 374, 366, 358, 327, 123, 134, 149, 198, 248, 249, 258, 293]
Engagingness =  [ 97, 477, 482, 444, 420, 372, 367, 317, 4, 122, 137, 148, 161, 180, 185, 189, 190, 209, 266, 287]
Memorability = [ 88, 73, 56, 90, 76, 489, 448, 395, 392, 389, 368, 350, 195, 299, 262, 282, 113, 160, 243, 617]
Specificity=[ 436, 356, 355, 333, 332, 94, 64, 567, 40, 133, 144, 177, 182, 194, 224, 234, 270, 276, 285, 547]
Criteria = Humanness + Engagingness + Memorability + Specificity
All = list(range(0, 1000))
All_Rand = [idx for idx in All if idx not in Criteria]


In [46]:
criteria = "Memorability"
df = pd.read_json('/home/uj-user/Yo/hybrid-ltm/data-80/context-fms-train/MS_DG/test.jsonl', lines=True, orient='records')
df = df[df['eval_indicator']!=criteria]
df.rename(columns={'input':'conversation', 'output':'response'}, inplace=True)
df.to_csv(f'context-fms-train-msdg-test-{criteria}.csv', index=False)

#### fms-bench와 평가를 위한 대조실험요 Random군 80개 생성

In [47]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

cc = load_dataset('jihyoung/ConversationChronicles')

/home/uj-user/Yo/hybrid-ltm/ltm-venv/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [48]:
df = cc['test'].to_pandas()
df_test = df.sample(frac=0.05, random_state=2023).reset_index()
df_test['eval_indicator'] = None
df_test.loc[Humanness, 'eval_indicator'] = 'Humanness'
df_test.loc[Engagingness, 'eval_indicator'] = 'Engagingness'
df_test.loc[Memorability, 'eval_indicator'] = 'Memorability'
df_test.loc[Specificity, 'eval_indicator'] = 'Specificity'
df_test

,index,dataID,relationship,time_interval,summary,first_session_dialogue,first_session_speakers,second_session_dialogue,second_session_speakers,third_session_dialogue,third_session_speakers,fourth_session_dialogue,fourth_session_speakers,fifth_session_dialogue,fifth_session_speakers,eval_indicator
0,8648,episode-150540,Neighbors,"[Start, A few days after, A few weeks after, A...",[Neighbors A and B catch up and discuss their ...,"[Hey there! It's so good to see you again!, We...","[Neighbors A, Neighbors B, Neighbors A, Neighb...",[Hey there! I was thinking about that problem ...,"[Neighbors A, Neighbors B, Neighbors A, Neighb...","[Hey, sorry about the other night. I got a bit...","[Neighbors A, Neighbors B, Neighbors A, Neighb...","[Hey, can I talk to you about something?, Of c...","[Neighbors A, Neighbors B, Neighbors A, Neighb...",[I'm so glad the exhibition went well. It was ...,"[Neighbors A, Neighbors B, Neighbors A, Neighb...",None
1,15597,episode-31403,Neighbors,"[Start, A few days after, A few days after, A ...",[Neighbors A crashes at Neighbors B's place du...,"[Hey, can I crash at your place tonight?, Sure...","[Neighbors A, Neighbors B, Neighbors A, Neighb...",[I just want you to know that I'm really picky...,"[Neighbors A, Neighbors B, Neighbors A, Neighb...",[That's a good idea. Thanks for suggesting it....,"[Neighbors A, Neighbors B, Neighbors A, Neighb...","[Hey, how are you doing today?, I'm doing pret...","[Neighbors A, Neighbors B, Neighbors A, Neighb...","[Hey, have you ever watched stand-up comedy ro...","[Neighbors A, Neighbors B, Neighbors A, Neighb...",None
2,6810,episode-120874,Co-workers,"[Start, A few weeks after, A few months after,...",[Two co-workers discuss improving their Englis...,"[Hey, B! I was impressed by your English skill...","[Co-workers A, Co-workers B, Co-workers A, Co-...","[Hey, how's it going?, Not too bad, how about ...","[Co-workers A, Co-workers B, Co-workers A, Co-...","[Hey, have you ever studied the Revolutionary ...","[Co-workers A, Co-workers B, Co-workers A, Co-...",[Can you believe the noise level in this offic...,"[Co-workers A, Co-workers B, Co-workers A, Co-...",[ I can't believe that happened again! My bloo...,"[Co-workers A, Co-workers B, Co-workers A, Co-...",None
3,17882,episode-210994,Co-workers,"[Start, A few days after, A few weeks after, A...",[Two co-workers discuss their preference for w...,"[Hey, do you like walking in the cold weather?...","[Co-workers A, Co-workers B, Co-workers A, Co-...","[Hey, it's good to see you again., Yeah, it's ...","[Co-workers A, Co-workers B, Co-workers A, Co-...",[I feel like I'm finally learning to go with t...,"[Co-workers A, Co-workers B, Co-workers A, Co-...",[I found a really nice place to stay! The owne...,"[Co-workers A, Co-workers B, Co-workers A, Co-...","[Hey, I just sent you the file for the meeting...","[Co-workers A, Co-workers B, Co-workers A, Co-...",None
4,3789,episode-161650,Patient and Doctor,"[Start, A couple of years after, A couple of y...",[The patient asks the doctor about the procedu...,"[ Hi, doctor. I'm all set for the surgery. Jus...","[Patient, Doctor, Patient, Doctor, Patient, Do...","[Doctor, that was amazing! How did you learn t...","[Patient, Doctor, Patient, Doctor, Patient, Do...","[Hello Patient, how can I assist you today?, H...","[Doctor, Patient, Doctor, Patient, Doctor, Pat...","[Doctor, I have to tell you about something th...","[Patient, Doctor, Patient, Doctor, Patient, Do...","[ Doctor, I'm feeling really jealous of my bro...","[Patient, Doctor, Patient, Doctor, Patient, Do...",Engagingness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5432,episode-180197,Co-workers,"[Start, A few weeks after, A few hours after, ...",[Co-worker A expresses frustration when they f...,[I feel like I always have to be doing multipl...,"[Co-workers A, Co-workers B, Co-workers A, Co-...",[Last night was so relaxing. I skipped making ...,"[Co-workers A, Co-workers B, Co-workers A, Co

In [50]:
from glob import glob
import os

dataset_path = glob('tasks/M*')
save_path = './tasks-rand'
os.makedirs(save_path, exist_ok=True)

for path in dataset_path:
    file_path = path + '/test.jsonl'
    dataset_name = path.split('/')[-1]
    print(file_path)
    df = pd.read_json(file_path, lines=True, orient='records')
    
    output = df['input'][All_Rand]#[df_test.dropna().index]
    output_df = pd.DataFrame(output)
    output_df['input'] = output_df['input'].map(lambda x: '```'+ '```'.join(x.split('```')[1:-3]) +'```')
    output_df['output'] = df['output'][All_Rand]
    output_df['eval_indicator'] = 'Rand'#df_test.dropna()['eval_indicator']
    print(output_df)
    output_df = output_df.sample(n=80, random_state=2023)#.reset_index()
    os.makedirs(f'{save_path}/{dataset_name}',exist_ok=True)
    output_df.to_json(f'{save_path}/{dataset_name}/test.jsonl', lines=True, orient='records')

tasks/MS_DG_SFT/test.jsonl
                                                 input  \
0    ```\nDialogue Session #1:\nNeighbors A:Hey the...   
1    ```\nDialogue Session #1:\nNeighbors A:Hey, ca...   
2    ```\nDialogue Session #1:\nCo-workers A:Hey, B...   
3    ```\nDialogue Session #1:\nCo-workers A:Hey, d...   
5    ```\nDialogue Session #1:\nPatient:Hi, Doctor....   
..                                                 ...   
995  ```\nDialogue Session #1:\nCo-workers A:I feel...   
996  ```\nDialogue Session #1:\nNeighbors A:Hey, ha...   
997  ```\nDialogue Session #1:\nClassmates A:Can yo...   
998  ```\nDialogue Session #1:\nNeighbors A:Hey, Ne...   
999  ```\nDialogue Session #1:\nClassmates A:I was ...   

                                                output eval_indicator  
0    Neighbors A:Well, I think we could try approac...           Rand  
1    Neighbors A:Absolutely. And I don't want to co...           Rand  
2    Co-workers A: Yeah, it's a nice distraction fr...      